Gộp dữ liệu     

In [ ]:
    # Cài đặt các thư viện
!pip install roboflow pyyaml tqdm ultralytics --quiet

In [ ]:


from roboflow import Roboflow
import os

# Sử dụng API key trực tiếp (chỉ dùng cho testing)
rf = Roboflow(api_key="YOUT-KEY-ON-ROBOFLOW")
workspace = rf.workspace("objdetecpen")

# Tải datasets
project_a = workspace.project("all-objects-1-z1qkf")
dataset_a = project_a.version(1).download("yolov8")

project_b = workspace.project("pen-x2n7a-x0key")
dataset_b = project_b.version(1).download("yolov8")

print("Download completed!")

In [ ]:
import os
import shutil

# Dataset paths trên Kaggle
dataset_a = "/kaggle/working/All-objects-1-1"
dataset_b = "/kaggle/working/pen-1" 
merged = "/kaggle/working/merged_with_eraser"

# Tạo thư mục
for split in ["train", "valid", "test"]:
    os.makedirs(os.path.join(merged, split, "images"), exist_ok=True)
    os.makedirs(os.path.join(merged, split, "labels"), exist_ok=True)

def copy_and_remap(src, dst, mapping):
    for split in ["train", "valid", "test"]:
        img_dir = os.path.join(src, split, "images")
        lbl_dir = os.path.join(src, split, "labels")
        if not os.path.exists(img_dir): continue

        # Copy ảnh
        for file in os.listdir(img_dir):
            shutil.copy(os.path.join(img_dir, file), os.path.join(dst, split, "images", file))

        # Remap nhãn
        for file in os.listdir(lbl_dir):
            lbl_src = os.path.join(lbl_dir, file)
            lbl_dst = os.path.join(dst, split, "labels", file)
            
            new_lines = []
            with open(lbl_src, "r") as f:
                for line in f:
                    cls, *coords = line.strip().split()
                    if int(cls) in mapping:
                        new_lines.append(f"{mapping[int(cls)]} {' '.join(coords)}")
            
            if new_lines:
                with open(lbl_dst, "w") as f:
                    f.write("\n".join(new_lines))

# Merge datasets
copy_and_remap(dataset_a, merged, {0:0, 1:1, 2:2})  # Dataset A
copy_and_remap(dataset_b, merged, {0:1})             # Dataset B

# Tạo file data.yaml
with open(os.path.join(merged, "data.yaml"), "w") as f:
    f.write(f"""path: {merged}
train: train/images
val: valid/images
test: test/images

names:
  0: eraser
  1: pen
  2: pencil
""")

print("✅ Dataset merged tại:", merged)

In [ ]:
import glob

for split in ["train", "valid", "test"]:
    imgs = glob.glob(f"{merged}/{split}/images/*.jpg")
    lbls = glob.glob(f"{merged}/{split}/labels/*.txt")
    print(f"{split}: {len(imgs)} images, {len(lbls)} labels")

# Đếm instance từng class
counts = {"eraser": 0, "pen": 0, "pencil": 0}
for split in ["train", "valid", "test"]:
    for lbl_file in glob.glob(f"{merged}/{split}/labels/*.txt"):
        with open(lbl_file, "r") as f:
            for line in f:
                cls = int(line.split()[0])
                if cls == 0:
                    counts["eraser"] += 1
                elif cls == 1:
                    counts["pen"] += 1
                elif cls == 2:
                    counts["pencil"] += 1

print("\nSố lượng instance sau merge:")
print(counts)

#train: 3211 images, 3209 labels
#valid: 689 images, 689 labels
#test: 334 images, 334 labels

#Số lượng instance sau merge:
#{'eraser': 916, 'pen': 2972, 'pencil': 2800}